In [1]:
!pip install azure-cognitiveservices-vision-computervision
!pip install --upgrade azure-cognitiveservices-vision-face

     |████████████████████████████████| 85 kB 4.2 MB/s eta 0:00:011
     |████████████████████████████████| 41 kB 2.0 MB/s  eta 0:00:01
     |████████████████████████████████| 151 kB 18.8 MB/s eta 0:00:01
     |████████████████████████████████| 67 kB 1.2 MB/s eta 0:00:01


In [4]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time


In [21]:
###HERE IS THE DOC
## https://docs.microsoft.com/en-us/azure/cognitive-services/computer-vision/quickstarts-sdk/image-analysis-client-library?tabs=visual-studio&pivots=programming-language-python


In [2]:
#subscription_key = "5b244e5d0ba74f388b7aa8ab6df63a3a"
#endpoint = "https://ashring.cognitiveservices.azure.com/"

#subscription_key = "6dfb399d95ea493d838973567604761b"
#endpoint = "https://oventery.cognitiveservices.azure.com/"

subscription_key = "777d01985bb34082b39fb54aa3d6a4b9"
endpoint = "https://dastermo.cognitiveservices.azure.com/"

In [5]:
## This is the image we need to Cycle through from Google Drive

remote_image_url = "https://upload.wikimedia.org/wikipedia/en/c/c7/Michael_Jordan_crying.jpg"

imgs = ['https://upload.wikimedia.org/wikipedia/commons/4/47/The_President%2C_Smt._Pratibha_Devisingh_Patil_with_the_President_of_UAE%2C_Sheikh_Khalifa_bin_Zayed_Al_Nahayan_at_the_Ceremonial_Reception%2C_at_Mushrif_Palace%2C_in_Abu_Dhabi_on_November_22%2C_2010_%28cropped%29.jpg',
        "https://upload.wikimedia.org/wikipedia/en/c/c7/Michael_Jordan_crying.jpg"]
imgs = ['Joyful-Girl-Model-Children-Face-Joy-Happy-5566390.jpeg', 
        'Madagascar_braids.jpeg',
        'af_dd.jpeg',
        'afro-afro-hair-beautiful-earings.jpeg',
        'andro.jpeg',
        'bald.jpeg',
        'context.jpeg',
        'down_syndromw.jpeg',
        'download.jpeg',
        'face_painting.jpeg',
        'holi.jpeg',
        'mask.jpeg',
        'shaven.jpeg']


In [6]:
for img in imgs:
  print(img)

Joyful-Girl-Model-Children-Face-Joy-Happy-5566390.jpeg
Madagascar_braids.jpeg
af_dd.jpeg
afro-afro-hair-beautiful-earings.jpeg
andro.jpeg
bald.jpeg
context.jpeg
down_syndromw.jpeg
download.jpeg
face_painting.jpeg
holi.jpeg
mask.jpeg
shaven.jpeg


In [7]:



computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

#remote_image_url = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-sample-data-files/master/ComputerVision/Images/landmark.jpg"

remote_image_url = "https://github.com/bsairesearch/faceconstruct/blob/main/faces/af_dd.jpeg"
def get_image_description():
    '''
    Describe an Image - remote
    This example describes the contents of an image with the confidence score.
    '''
    print("===== Describe an image - remote =====")
    # Call API
    description_results = computervision_client.describe_image(remote_image_url )

    # Get the captions (descriptions) from the response, with confidence level
    print("Description of remote image: ")
    if (len(description_results.captions) == 0):
        print("No description detected.")
    else:
        for caption in description_results.captions:
            print("'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100))
            res.append("'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100))

    return res

def get_image_category():
    '''
    Categorize an Image - remote
    This example extracts (general) categories from a remote image with a confidence score.
    '''
    print("===== Categorize an image - remote =====")
    # Select the visual feature(s) you want.
    remote_image_features = ["categories"]
    # Call API with URL and features
    categorize_results_remote = computervision_client.analyze_image(remote_image_url, remote_image_features)

    # Print results with confidence score
    print("Categories from remote image: ")
    if (len(categorize_results_remote.categories) == 0):
        print("No categories detected.")
    else:
        for category in categorize_results_remote.categories:
            print("'{}' with confidence {:.2f}%".format(category.name, category.score * 100))

    return categorize_results_remote

def get_image_tags():
    '''
    Tag an Image - remote
    This example returns a tag (key word) for each thing in the image.
    '''
    print("===== Tag an image - remote =====")
    # Call API with remote image
    tags_result_remote = computervision_client.tag_image(remote_image_url)

    # Print results with confidence score
    print("Tags in the remote image: ")
    if (len(tags_result_remote.tags) == 0):
        print("No tags detected.")
    else:
        for tag in tags_result_remote.tags:
            print("'{}' with confidence {:.2f}%".format(tag.name, tag.confidence * 100))

    return tags_result_remote

def detect_objects():
    '''
    Detect Objects - remote
    This example detects different kinds of objects with bounding boxes in a remote image.
    '''
    print("===== Detect Objects - remote =====")
    # Get URL image with different objects
    remote_image_url_objects = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-sample-data-files/master/ComputerVision/Images/objects.jpg"
    # Call API with URL
    tags_result_remote = computervision_client.detect_objects(remote_image_url_objects)

    # Print detected objects results with bounding boxes
    print("Detecting objects in remote image:")
    if len(detect_objects_results_remote.objects) == 0:
        print("No objects detected.")
    else:
        for object in detect_objects_results_remote.objects:
            print("object at location {}, {}, {}, {}".format( \
                object.rectangle.x, object.rectangle.x + object.rectangle.w, \
                object.rectangle.y, object.rectangle.y + object.rectangle.h))
    return tags_result_remote

def detect_faces():
    '''
    Detect Faces - remote
    This example detects faces in a remote image, gets their gender and age, 
    and marks them with a bounding box.
    '''
    print("===== Detect Faces - remote =====")
    # Get an image with faces
    remote_image_url_faces = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-sample-data-files/master/ComputerVision/Images/faces.jpg"
    # Select the visual feature(s) you want.
    remote_image_features = ["faces"]
    # Call the API with remote URL and features
    detect_faces_results_remote = computervision_client.analyze_image(remote_image_url_faces, remote_image_features)

    # Print the results with gender, age, and bounding box
    print("Faces in the remote image: ")
    if (len(detect_faces_results_remote.faces) == 0):
        print("No faces detected.")
    else:
        for face in detect_faces_results_remote.faces:
            print("'{}' of age {} at location {}, {}, {}, {}".format(face.gender, face.age, \
            face.face_rectangle.left, face.face_rectangle.top, \
            face.face_rectangle.left + face.face_rectangle.width, \
            face.face_rectangle.top + face.face_rectangle.height))
    return detect_faces_results_remote
    


In [49]:

loc = 'https://raw.githubusercontent.com/bsairesearch/faceconstruct/main/faces/'
for img in imgs:
  remote_image_url = loc + img
  print("\n\n######## URL = " + remote_image_url)
  get_image_description()
  get_image_category()
  get_image_tags()
  #detect_objects()
  detect_faces()

print("\n\n#####DONE")




######## URL = https://raw.githubusercontent.com/bsairesearch/faceconstruct/main/faces/Joyful-Girl-Model-Children-Face-Joy-Happy-5566390.jpeg
===== Describe an image - remote =====
Description of remote image: 
'a girl with a flower in her hair' with confidence 43.69%
===== Categorize an image - remote =====
Categories from remote image: 
'people_baby' with confidence 53.52%
'people_young' with confidence 41.80%
===== Tag an image - remote =====
Tags in the remote image: 
'person' with confidence 99.79%
'human face' with confidence 99.46%
'smile' with confidence 96.74%
'clothing' with confidence 96.36%
'fashion accessory' with confidence 96.05%
'toddler' with confidence 95.85%
'child' with confidence 90.11%
'happy' with confidence 89.79%
'tooth' with confidence 87.48%
'indoor' with confidence 87.36%
'necklace' with confidence 86.91%
'skin' with confidence 86.52%
'cheek' with confidence 86.26%
'baby' with confidence 86.14%
'finger' with confidence 84.52%
'wall' with confidence 73.65%


In [33]:
resp = []

res=[]
res.append("a")
res.append("b")
resp.append(res)


print(str(resp))

[['a', 'b']]


In [19]:
import os

os.getcwd()
##os.curdir


'/Users/suresh/dev/FaceConstruct/faceconstruct'

In [ ]:

import os


loc = 'https://raw.githubusercontent.com/bsairesearch/faceconstruct/main/faces/'



for img in imgs:
  remote_image_url = loc + img
  print("\n\n######## URL = " + remote_image_url)
  get_image_description()
  get_image_category()
  get_image_tags()
  #detect_objects()
  detect_faces()

print("\n\n#####DONE")